In [26]:
from pytubefix import YouTube
from pytubefix.cli import on_progress

url = 'https://www.youtube.com/watch?v=-jRiPImDF3w'
yt = YouTube(url, on_progress_callback=on_progress)

def itags():
    max_audio = 0
    audio_value = 0
    for audio_stream in yt.streams.filter(only_audio=True):
        abr = int(audio_stream.abr.replace('kbps', ''))
        if abr > max_audio:
            max_audio = abr
            audio_value = audio_stream.itag

    return audio_value


audio = itags()
audio_file = yt.streams.get_by_itag(audio).download()

In [27]:
import os
from google.cloud import speech

# Configurar credenciales de Google Cloud
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "dsma-410321-dfea94bcf18f.json"

def transcribe_audio(audio_file):
    client = speech.SpeechClient()

    # Cargar el archivo de audio
    with open(audio_file, "rb") as audio:
        content = audio.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.MP3,
        sample_rate_hertz=44100,
        language_code="es-ES"
    )

    # Realizar la transcripción
    response = client.recognize(config=config, audio=audio)

    results = []

    # Procesar y mostrar los resultados
    for result in response.results:
        results.append(result.alternatives[0].transcript)
        print(f"Transcripción: {result.alternatives[0].transcript}")

    return results

In [28]:
transcripts = transcribe_audio(audio_file)

KeyboardInterrupt: 